In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.parser import parse
from time import sleep, time
from random import randint
import re
from IPython.core.display import clear_output
from warnings import warn
import numpy as np
from selenium import webdriver 
import cv2 
import pytesseract
from PIL import Image

In [2]:
def is_captcha_page(soup):
    captcha_text="Please verify you are human by typing the text below and clicking submit."
    page_info=soup.find('font',attrs={'class':"pagebodybold"})
    if page_info!=None and captcha_text in page_info.text:
        return True
    return False

In [3]:
def solve_captcha(company_link):
    driver = webdriver.Firefox('/home/udit/projects/chromiumm')
    driver.get(company_link)
    element = driver.find_element_by_id("imgCaptcha")
    element.screenshot('final.png')
    img = cv2.imread('final.png')
    captcha=pytesseract.image_to_string(img)
    
    captcha=re.sub(r'\W+', '', captcha)
    captcha_input = driver.find_element_by_xpath("//input[@id ='txtCaptcha']") 
    captcha_input.send_keys(captcha) 

    # submit button clicked
    driver.find_element_by_xpath("//input[@id ='btnRegiser']").click()
    html = driver.page_source
    soup = BeautifulSoup(html)
    driver.close()
    return soup

In [4]:
def try_filling(strr,company_info):
    try:
        global x
        eqn= "{0} = {1}".format("x",strr)
        x = np.nan
        exec(eqn,globals())
        return x
    except IndexError:
        return(np.nan)
    except ValueError:
        return(np.nan)

In [5]:
URL = "https://ibanknet.com/scripts/callreports/fiList.aspx?type=bankmanda"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

In [6]:
companies=soup.find_all('a', attrs = {'class':'pagebody'}) 

In [ ]:
all_companies=[]
error_links=[]

for company in companies:
    company_link="https://ibanknet.com/"+company.get('href')
    try:
        c = requests.get(company_link)
        soup = BeautifulSoup(c.content, 'html5lib')

        while(is_captcha_page(soup)):
            soup=solve_captcha(company_link)

        company_info=[]

        for info in soup.find_all('td', attrs = {'class':'instinfo'}):
            company_info.append(info.text)


        company_inf={}
        company_inf["company_name"]=soup.find('font',attrs={'class':"blockuhilite"}).text.strip()
        company_inf["RSSD_ID"]=try_filling('company_info[0].split()[company_info[0].split().index("RSSD")+2]',company_info)
        company_inf['RefinedAddress']=try_filling("company_info[1]",company_info)
        company_inf['City']=try_filling('company_info[4].split(",")[0].strip()',company_info)
        company_inf['State']=try_filling('company_info[4].split(",")[1].strip()',company_info)
        company_inf['Pin']=try_filling('company_info[4].split(",")[2].strip()',company_info)
        company_inf['webAddress']=try_filling("company_info[7].strip() if ('.com' in company_info[7].strip()) else np.nan",company_info)
        company_inf['YearOpened']=try_filling("company_info[company_info.index('Year Opened : ')+1]",company_info)
        company_inf['FiduciaryPowers']=try_filling("company_info[company_info.index('Fiduciary Powers : ')+1]",company_info)
        company_inf['EntityType']=try_filling("company_info[company_info.index('Entity Type : ')+1]",company_info)
        company_inf['PowersExercised']=try_filling("company_info[company_info.index('Powers Exercised : ')+1]",company_info)
        company_inf['MemberFederalReserve']=try_filling("company_info[company_info.index('Member Federal Reserve : ')+1]",company_info)
        company_inf['InstitutionType']=try_filling("company_info[company_info.index('Institution Type : ')+1]",company_info)
        company_inf['FederalRegulator']=try_filling("company_info[company_info.index('Federal Regulator : ')+1]",company_info)
        company_inf['S-Corp']=try_filling("company_info[company_info.index('S-Corp : ')+1]",company_info)
        company_inf['Specialization']=try_filling("company_info[company_info.index('Specialization : ')+1]",company_info)

        all_companies.append(company_inf)
        
    except Exception as e:
        print(str(e))
        error_links.append([company_link,str(e)])

In [ ]:
all_compaies=pd.DataFrame(all_companies)

In [ ]:
all_compaies.head()

In [ ]:
error_links